<a href="https://colab.research.google.com/github/sogangori/choongang20/blob/master/cnn_2_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2 자리수 숫자 CNN 분류기 

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

In [5]:
x, y = load_digits(return_X_y=True)
x.shape, y.shape, set(y)

((1797, 64), (1797,), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

공유 : http://bitly.kr/1CC8BWNm

In [0]:
 xx0 = np.concatenate((x[0].reshape((8,8)), x[1].reshape((8,8))), axis=1)#축1 방향으로 붙이기
 xx1 = np.concatenate((x[2].reshape((8,8)), x[3].reshape((8,8))), axis=1)
 yy0 = y[:2]
 yy1 = y[2:4]
 #plt.title(str(yy0))
 #plt.imshow(xx0)

In [16]:
x_train = np.stack((xx0, xx1), 0)
y_train = np.stack((yy0, yy1), 0)
x_train.shape, y_train.shape

((2, 8, 16), (2, 2))

In [0]:
class MyModel(keras.Model): # github.com/sogangori/choongang20/keras_cnn_expert.ipynb 
  def __init__(self):
    super(MyModel, self).__init__()
    self.opt = tf.optimizers.SGD(learning_rate=0.01)#Stochatic Gradient Descent 확률적 경사 하강
    self.conv0 = keras.layers.Conv2D(16, [3,3], padding='same', activation=keras.activations.relu)
    self.conv1 = keras.layers.Conv2D(32, [3,3], padding='same', activation=keras.activations.relu)
    self.pool0 = keras.layers.MaxPool2D([2,2], padding='same')
    self.pool1 = keras.layers.MaxPool2D([2,2], padding='same')
    self.flatten = keras.layers.Flatten()
    self.dense = keras.layers.Dense(units=10 * 2)
  
  def call(self, x):
    #x (1797, 64)
    x_4d = tf.reshape(x, [-1,8,8*2,1]) 
    x_4d = tf.cast(x_4d, tf.float32)
    net = self.conv0(x_4d)
    net = self.pool0(net)
    net = self.conv1(net)
    net = self.pool1(net)
    net = self.flatten(net)    
    h = self.dense(net)
    h = tf.reshape(h, [-1, 2, 10]) # 2:두자리수, 10:10개의 클래스 
    h = tf.nn.softmax(h, axis=2)
    return h

  def get_loss(self, y, h):
    #학습할때 nan이 발생하는 경우 값을 clip(자르다) (최소값, 최대값) 
    h = tf.clip_by_value(h, 1e-8, 1 - 1e-8) # h 가 0이나 1이 되지 않도록 하는 안전장치 
    cross_entropy = - (y * tf.math.log(h) + (1 - y) * tf.math.log(1 - h)) 
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):    
    predict = tf.argmax(h, -1)
    self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32)) # True > 1, False > 0 로 cast

  def fit(self, x, y, epoch=1):
    # x : (m, 8, 16), y: (m, 2)    
    y_hot = tf.one_hot(y, depth=10, axis=-1)#(m, 2, 10)  
    for i in range(epoch):
      with tf.GradientTape() as tape: #경사 기록 장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)        
      grads = tape.gradient(loss, self.trainable_variables) #경사 계산
      self.opt.apply_gradients(zip(grads, self.trainable_variables)) # 가중치에서 경사를 빼기
      self.get_accuracy(y, h)
      if i%10==0:
        print('%d/%d loss:%.3f acc:%.3f'%(i, epoch, loss, self.acc))
model = MyModel()

In [21]:
model.fit(x_train, y_train, epoch=100)

0/100 loss:0.175 acc:0.750
10/100 loss:0.051 acc:1.000
20/100 loss:0.026 acc:1.000
30/100 loss:0.018 acc:1.000
40/100 loss:0.013 acc:1.000
50/100 loss:0.011 acc:1.000
60/100 loss:0.009 acc:1.000
70/100 loss:0.008 acc:1.000
80/100 loss:0.007 acc:1.000
90/100 loss:0.006 acc:1.000
